This notebook allows you to mannually and interactively estimate an approximate value of FFH from a given GSV pano image, which is useful for ground-truth data assessment and updates (if required) and more reliable than visual assessment. It requires:
* building height (or eave height) measured from building & canopy height raster
* pano id of corresponding building (to allow searching for pano image)
* interactively measured building height (or eave height) in pixels
* interactively measured floor height in pixels

### Load dependencies

In [40]:
import cv2
import glob
import os

### Set input parameters:
* input pano images folder
* pano image id
* building/eave height (m) - derived from building/canopy height raster

In [41]:
pano_folder=r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\Data Exploration\GSV\Wagga\Pano_clipped_all"


In [42]:
# pano_id='Sdp4ax7QS6z-NBY_iBH5ww'
# building_height_m=3.5

pano_id='qRwIoP-OssOI6-Czcvn9Xg'
building_height_m=3.5

In [43]:
# Load image
pano_file=glob.glob(os.path.join(pano_folder,pano_id+'.jpg'))[0]
original_img = cv2.imread(pano_file)

### Store measured points

In [44]:
original_img_copy=original_img.copy()

# List to store two points
points_buildingheight = []
points_floorheight = []
dist_buildingheight=[]
dist_floorheight=[]


### defines callback function

In [45]:
def measure_distance(event, x, y, flags, param):
    """Handles mouse clicks to measure X and Y distances."""
    original_img, points, dists, window_name = param  # Unpack parameters
    # global points, img

    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))

        if len(points) == 2:
            # Calculate distances
            x_dist = abs(points[1][0] - points[0][0])
            y_dist = abs(points[1][1] - points[0][1])

            print(f"Start point coordinates: {points[0]}")
            print(f"End point coordinates: {points[1]}")
            print(f"X Distance: {x_dist} pixels")
            print(f"Y Distance: {y_dist} pixels")
            
            dists.append((x_dist,y_dist))

            # Draw points and line
            img = original_img.copy()  # Reset image
            cv2.circle(img, points[0], 5, (0, 0, 255), -1)  # Red point
            cv2.circle(img, points[1], 5, (0, 255, 0), -1)  # Green point
            cv2.line(img, points[0], (points[1][0], points[0][1]), (255, 0, 0), 2)  # Horizontal line
            cv2.line(img, points[1], (points[1][0], points[0][1]), (255, 0, 0), 2)  # Vertical line
            cv2.imshow(window_name, img)

            points = []  # Reset points
            dists = []

### Interactively measure building/eave and floor heights
* single click on two points on the image
* press esc key once finished

In [46]:
# Create two windows
window_buildingheight='Measure_building_height_'+pano_id
window_floorheight='Measure_floor_height_'+pano_id
cv2.namedWindow(window_buildingheight)
cv2.namedWindow(window_floorheight)

# set mouse callback
cv2.setMouseCallback(window_buildingheight, measure_distance, param=(original_img, points_buildingheight, dist_buildingheight,window_buildingheight))
cv2.setMouseCallback(window_floorheight, measure_distance, param=(original_img, points_floorheight, dist_floorheight,window_floorheight))

# Show image
while True:
    # Display the first window
    cv2.imshow(window_buildingheight, original_img)
    # Wait for a key press
    key = cv2.waitKey(0) & 0xFF  # Wait indefinitely until a key is pressed
    # Check if ESC key is pressed (27 is the ASCII value of ESC)
    if key == 27:  # ESC key
        cv2.destroyWindow(window_buildingheight)  # Close the first window
        break  # Exit the loop when the first window is closed

while True:
    cv2.imshow(window_floorheight, original_img)
    key = cv2.waitKey(0) & 0xFF
    if key == 27:  # ESC key
        cv2.destroyWindow(window_buildingheight)
        break

# Close all windows when both are closed
cv2.destroyAllWindows()

Start point coordinates: (1642, 1978)
End point coordinates: (1670, 2294)
X Distance: 28 pixels
Y Distance: 316 pixels
Start point coordinates: (1373, 2248)
End point coordinates: (1379, 2309)
X Distance: 6 pixels
Y Distance: 61 pixels


### Estimate floor height

In [47]:
floor_height_m=dist_floorheight[0][1]/dist_buildingheight[0][1]*building_height_m
print('Estimated floor height (m):',floor_height_m)

Estimated floor height (m): 0.6756329113924051
